In [143]:
import re
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sentence_transformers import SentenceTransformer

In [144]:
import warnings
warnings.filterwarnings('ignore')

In [145]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [146]:
df = pd.read_csv('joe_biden_multiple_news_OR.csv')
df.head()

,url,title,description,maintext,date_publish
0,https://www.cnn.com/2024/01/22/politics/fake-j...,Fake Joe Biden robocall urges New Hampshire vo...,A robocall that appears to be an AI voice rese...,CNN —\nA robocall that appears to be an AI voi...,2024-01-22 17:13:52
1,https://www.nytimes.com/2020/05/22/us/politics...,Biden Apologizes for Saying Black Voters ‘Ain’...,The former vice president ignited controversy ...,"“I shouldn’t have been such a wise guy,” Mr. B...",2020-05-22 14:41:50
2,https://www.cnn.com/videos/politics/2020/05/22...,Joe Biden says voters ‘ain’t black’ if they su...,Joe Biden apologized hours after he told a pop...,1. How relevant is this ad to you?\nVideo play...,2020-05-22 22:26:50
3,https://www.cnn.com/2020/05/22/politics/biden-...,Biden: ‘If you have a problem figuring out whe...,Joe Biden said he “shouldn’t have been such a ...,CNN —\nJoe Biden said he “shouldn’t have been ...,2020-05-22 15:17:54
4,https://www.bbc.com/news/world-us-canada-68064247,Fake Biden robocall tells voters to skip New H...,"An investigation is opened into the calls, whi...",Fake Biden robocall tells voters to skip New H...,2024-01-22 23:39:43


In [147]:
def split_into_sentences(text):
    # Define the regex pattern for sentence splitting
    sentence_pattern = r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s'

    # Split the text into sentences using the pattern
    sentences = re.split(sentence_pattern, text)

    # Print each sentence
    return sentences

In [148]:
sentences = df.maintext.apply(lambda x: x.split('\n')) # df.maintext.apply(split_into_sentences)
sentences = [sentence for lst in sentences for sentence in lst]

In [149]:
numbers_of_sentences = df.maintext.apply(lambda x: x.split('\n')).apply(len)

sentence_to_id = []
for i, num in enumerate(numbers_of_sentences):
    sentence_to_id += [i] * num

In [150]:
embeddings = model.encode(sentences)

In [151]:
nn = NearestNeighbors(n_neighbors=3)
nn.fit(embeddings)

NearestNeighbors(n_neighbors=3)

In [152]:
query = 'Did Joe Biden tell people not to vote?'

In [153]:
target = model.encode(query)

In [154]:
indices = nn.kneighbors(target[np.newaxis, :], return_distance=False)

In [155]:
news_indices = np.array(sentence_to_id)[indices]

In [156]:
relevant = np.array(sentences)[indices]

In [157]:
output = pd.DataFrame({
    'title': df.title.iloc[list(news_indices[0])].to_list(),
    'sen': relevant[0]
})

In [158]:
context = []
for i in range(output.shape[0]):
    title = output.at[i, 'title']
    sen = output.at[i, 'sen']
    
    context.append(
        f'source: {title}\n{sen}'
    )

In [159]:
context = '\n'.join(context)

In [160]:
prompt = f"""Based only on the context:
{context}
Tell me if this statement is a fact: {query}
Your replying format: <Your answer> <evidence in simple words with source>"""

In [161]:
print(prompt)

Based only on the context:
source: Biden’s Gaza problems are deeper than just being interrupted in public
“I believe he is beyond redemption,” said Khalid Turaani, 57, a consultant based in Michigan and co-chair of the state’s chapter of the Abandon Biden movement. “I will not vote for Joe Biden. I believe that his complicity and his active participation in the genocide against the Palestinian people in Gaza disqualify him from my vote.”
source: Fake Joe Biden robocall urges New Hampshire voters not to vote in Tuesday’s Democratic primary
“It’s very apparent that there’s someone, or someones, out there who want to suppress the primary write-in vote on Tuesday, tomorrow, by sending out this robocall saying, ‘Don’t vote for Joe Biden,’ or Joe Biden saying, ‘Don’t vote on Tuesday, save your vote until November,’” said Sullivan. “It’s voter suppression by someone who wants to hurt Joe Biden.”
source: Biden slams GOP vote-restriction bills as ‘sick’ and ‘un-American’ while Georgia moves to 

In [163]:
print("""alse. There is no evidence in the provided context that Joe Biden explicitly told people not to vote. The statement referencing not voting for Joe Biden comes from a fake robocall urging New Hampshire voters not to vote for him. Additionally, there is no indication that Biden himself made such a statement. Therefore, based on the evidence provided, the statement that Joe Biden told people not to vote is not factual.""")

alse. There is no evidence in the provided context that Joe Biden explicitly told people not to vote. The statement referencing not voting for Joe Biden comes from a fake robocall urging New Hampshire voters not to vote for him. Additionally, there is no indication that Biden himself made such a statement. Therefore, based on the evidence provided, the statement that Joe Biden told people not to vote is not factual.
